In [1]:
# ! pip install gym

In [2]:
import gym
import creversi.gym_reversi
from creversi import *
from itertools import count
import math
import random
import numpy as np
import copy

GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 2000
ALPHA = 0.3

env = gym.make('Reversi-v0').unwrapped

/home/nkmur/.pyenv/versions/3.9.7/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (8, 8)
  logger.warn(


In [3]:
class QFunc:
    def __init__(self, alpha=ALPHA, gamma=GAMMA, initial_q = 1.0):
        self._initial_q = initial_q
        self._values = {}
        self._alpha = alpha
        self._gamma = gamma

    def get(self, state, act):
        if self._values.get((state, act)) is None:
            # self._values[(state, act)] = self._initial_q
            return self._initial_q
        return self._values.get((state, act))

    def set(self, s, a, q_value):
        self._values[s, a] = q_value

    def update(self, s, a, r, max_q):
        # print(a)
        pQ = self.get(s, a)
        new_q = pQ + self._alpha * ((r + self._gamma * max_q) - pQ)
        # print(s)
        # print(new_q)
        self.set(s, a, new_q)



In [6]:
class QAgent:
    def __init__(self):
        self.q = QFunc()
        self.name = "QA"
        # self.action_count = 0
        self.last_state = ()
        self.last_move = -1
    def select_move(self,board, episode):
        return self.policy(board, episode)

    def get_bitboard(self,board):
        bitboard = np.empty(1, creversi.dtypeBitboard)
        board.to_bitboard(bitboard)
        state = tuple(bitboard[0])#ndarray->hashableに変換
        return state
    
    def policy(self,board, episode):#方策
        positions = list(board.legal_moves)
        state = self.get_bitboard(board)
        # if len(positions) == 0:
        #     return "pass"

        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * episode / EPS_DECAY)
        if sample < eps_threshold:
            move = random.choice(positions)
        else:
            qs = []
            for position in positions:
                # qs.append(self.q.get(tuple(self._last_board.flattend_data()), position))#候補手毎のQ値
                qs.append(self.q.get(state, position))

            max_q = max(qs) ##最も評価が高い手

            if qs.count(max_q) > 1:
                # more than 1 best option; choose among them randomly
                best_options = [i for i in range(len(positions)) if qs[i] == max_q]
                i = random.choice(best_options)
            else:
                i = qs.index(max_q)
            move = positions[i]
        self.last_state = state
        self.last_move = move
        return move
    

    def learn(self, board, move, reward, next_board, done, max_next_q):#Q学習
        state = self.get_bitboard(board)
        # next_q_list = []
        # next_state = self.get_bitboard(next_board)
        # next_moves = list(next_board.legal_moves)
        # for legal_next_move in next_moves:
        #     next_q_list.append(self.q.get(next_state, legal_next_move))

        # if done or len(next_q_list) == 0:
        #     max_q_new = 0
        # else:
        #     max_q_new = max(next_q_list)
        # max_q_new = -max_q_new
        self.q.update(self.last_state, self.last_move, reward, max_next_q)


class RandomAgent:
    def __init__(self):
        # self.board = board
        self.last_board = None
        self.last_move = None
        self.name = "RA"
    def select_move(self, board, episode):
        list_move = list(board.legal_moves)
        # rand_move_num = random.randint(0,len(list_move)-1)
        move = random.choice(list_move)
        return move
        

In [8]:
def result_process(results, board ,episode):
        if board.turn == BLACK_TURN:
            piece_nums = [board.piece_num(), board.opponent_piece_num()]
        else:
            piece_nums = [board.opponent_piece_num(), board.piece_num()]

        # print(f'result black={piece_nums[0]} white={piece_nums[1]}')
        if piece_nums[0] > piece_nums[1]:
            # print('black won')
            results[0]+=1
        elif piece_nums[1] > piece_nums[0]:
            # print('white won')
           results[1]+=1
        else:
            # print('draw')
            results[2]+=1
            
        if episode%500==0:
            print(f"Black:{results[0]} White:{results[1]} even:{results[2]}")

## 学習

In [10]:
def get_max_next_q(agent, next_board, done):
    next_q_list = []
    next_state = agent.get_bitboard(next_board)
    next_moves = list(next_board.legal_moves)
    for legal_next_move in next_moves:
        next_q_list.append(agent.q.get(next_state, legal_next_move))

    if done or len(next_q_list) == 0:
        max_next_q = 0.0
    else:
        max_next_q = max(next_q_list)
    max_next_q = max_next_q
    return max_next_q

In [17]:
num_episodes = 10000
episodes_done = 0
Black = QAgent()
# White = QAgent() 
White = RandomAgent()
results=[0,0,0]
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 2000
max_next_q = 0
for i_episode in range(num_episodes):
    env.reset()
    # last_move = 64
    # last_board = None
    # reward = 0
    # done = False
    b = 0
    w = 0
    for t in count():
        
        b_move = Black.select_move(env.board, i_episode)
        b_board = copy.copy(env.board)#石を打つ前に状態保存
        # print(board)
        b_next_board, reward, done, is_draw = env.step(b_move)
        # print(board)
        # print(env.board)
        # print(f"b_reward={reward}")
        # print(f"b_move={b_move}")
        # print(f"Black.last_move={Black.last_move}")

        if White.name == "QA" and White.last_move in range(64):
            # max_next_q = get_max_next_q(Black, next_board, done)
            max_next_q = get_max_next_q(White, b_next_board, done)
            White.learn(w_board, w_move, reward, b_next_board, done ,max_next_q)
            w += 1
            # print(f"w = {w}")
        
        if done and Black.last_move==64:
            break

        w_move = White.select_move(env.board, i_episode)
        w_board = copy.copy(env.board)
        w_next_board, reward, done, is_draw = env.step(w_move)
        # print(f"w_reward={reward}")
        # print(f"White.last_move={White.last_move}")

        if Black.name == "QA" and Black.last_move in range(64):#pass->64
            # max_next_q = get_max_next_q(White, next_board, done)
            max_next_q = get_max_next_q(Black, w_next_board, done)
            Black.learn(b_board, b_move, reward, w_next_board, done,max_next_q)
            b += 1
            # print(f"b = {b}")

        if done and White.last_move==64:
            break
    # record_result(results, env.board, i_episode)
    result_process(results, env.board, i_episode)
    episodes_done += 1



Black:1 White:0 even:0
Black:240 White:237 even:24
Black:460 White:499 even:42
Black:690 White:740 even:71
Black:939 White:972 even:90
Black:1159 White:1234 even:108
Black:1387 White:1484 even:130
Black:1624 White:1728 even:149
Black:1834 White:1996 even:171
Black:2070 White:2241 even:190
Black:2288 White:2499 even:214
Black:2524 White:2749 even:228
Black:2750 White:2994 even:257
Black:2973 White:3247 even:281
Black:3210 White:3491 even:300
Black:3435 White:3746 even:320
Black:3645 White:4005 even:351
Black:3896 White:4228 even:377
Black:4120 White:4493 even:388
Black:4350 White:4736 even:415


## 性能評価

### Black

In [18]:
num_episodes = 10000
episodes_done = 0
# Black = QAgent()
White_R = RandomAgent()
results=[0,0,0]
EPS_START = 0
EPS_END = 0
EPS_DECAY = 1
for i_episode in range(num_episodes):
    env.reset()
    
    for t in count():

        b_move = Black.select_move(env.board, i_episode)
        next_board, reward, done, is_draw = env.step(b_move)

        w_move = White_R.select_move(env.board, i_episode)
        next_board, reward, done, is_draw = env.step(w_move)

        if done:
            break
    result_process(results, env.board, i_episode)
    episodes_done += 1

Black:1 White:0 even:0
Black:231 White:252 even:18
Black:456 White:505 even:40
Black:668 White:770 even:63
Black:880 White:1036 even:85
Black:1114 White:1281 even:106
Black:1334 White:1543 even:124
Black:1553 White:1799 even:149
Black:1767 White:2060 even:174
Black:2017 White:2288 even:196
Black:2233 White:2543 even:225
Black:2456 White:2791 even:254
Black:2685 White:3041 even:275
Black:2922 White:3287 even:292
Black:3161 White:3532 even:308
Black:3383 White:3795 even:323
Black:3612 White:4044 even:345
Black:3838 White:4301 even:362
Black:4075 White:4543 even:383
Black:4298 White:4799 even:404


### White

In [ ]:
num_episodes = 10000
episodes_done = 0
# Black = QAgent()
Black_R = RandomAgent()
results=[0,0,0]
EPS_START = 0
EPS_END = 0
EPS_DECAY = 1
for i_episode in range(num_episodes):
    env.reset()
    
    for t in count():

        b_move = Black_R.select_move(env.board, i_episode)
        next_board, reward, done, is_draw = env.step(b_move)

        w_move = White.select_move(env.board, i_episode)
        next_board, reward, done, is_draw = env.step(w_move)

        if done:
            break
    result_process(results, env.board, i_episode)
    episodes_done += 1

Black:0 White:1 even:0
Black:217 White:264 even:20
Black:442 White:519 even:40
Black:682 White:757 even:62
Black:895 White:1025 even:81
Black:1121 White:1271 even:109
Black:1338 White:1534 even:129
Black:1543 White:1806 even:152
Black:1765 White:2067 even:169
Black:1993 White:2319 even:189
Black:2223 White:2569 even:209
Black:2457 White:2811 even:233
Black:2694 White:3054 even:253
Black:2917 White:3311 even:273
Black:3142 White:3558 even:301
Black:3389 White:3797 even:315
Black:3593 White:4074 even:334
Black:3812 White:4339 even:350
Black:4047 White:4584 even:370
Black:4275 White:4827 even:399
